# Validate Analysis Workflow Notebook
This notebook is a local version of the Google Colab-hosted analysis workflow notebook for MapSwipe Validate projects. This is for local integration testing of notebook / codebase interactions.

In [ ]:
%load_ext autoreload
%autoreload 2

# Step 1: Analyze Project

Replace the `project_id` value in the cell below with the ID of your project. Then open the Outline tab on the left-hand side of the screen, right-click on the `Step 1: Analyze Project` header, and choose `Select and Run Cell(s) for this Heading`.

In [ ]:
# project_id = "-O7hFcC2pKTnTh01SGds"  # TODO choose a more friendly default
project_id = "-O2_J_0_ChfONXhYeRoU"
#project_id = "-O6MPjXrwBGIbLm1pWro"

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from mapswipe.workflows.project_remap import analyze_project
analysis_results = analyze_project(project_id)

# Step 2: Visualize Results

In [ ]:
from mapswipe.workflows.viz import create_moran_quad_map, create_moran_quad_hex_map, create_task_map
from mapswipe.workflows.project_remap import MORAN_DISTANCE_METERS, DEPENDENT_VAR, MODEL_FEATURES, HEX_VIZ_H3_RESOLUTION

In [ ]:
analysis_results["df_avg_remap"]

In [ ]:
df_agg_moran_w = analysis_results["df_agg_moran_w"]
#moran_col = f"moran_quad_{int(MORAN_DISTANCE_METERS)}m"
#df_agg_moran_w[moran_col] = df_agg_moran_w[moran_col].apply(lambda x: x if x == 1 else 0)
#create_moran_quad_hex_map(df_agg_moran_w, f"moran_quad_{int(MORAN_DISTANCE_METERS)}m", value_cols=MODEL_FEATURES + ["remap_score", DEPENDENT_VAR], h3_resolution=HEX_VIZ_H3_RESOLUTION)
#create_moran_quad_map(df_agg_moran_w, f"moran_quad_{int(MORAN_DISTANCE_METERS)}m", value_cols=MODEL_FEATURES + ["remap_score", DEPENDENT_VAR], include_legend=False)
create_task_map(df_agg_moran_w, "remap_score", ["task_id", "remap_score", "0_share_uw"], {"remap_score": "Remap Score", "0_share_uw": "Raw % 'No' Responses", "task_id": "Task ID"})

# Step 3: Set Thresholds

In [ ]:
threshold_N = 5 # @param {type:"number"}
#threshold_type = "Top N Tasks" # @param ["Top N Tasks", "Top N% of Tasks", "Tasks With >=N remap_score"]
threshold_type = "Top N% of Tasks"

In [ ]:
from mapswipe.workflows.project_remap import SELECTION_COL, apply_threshold_filter
gdf_threshold = apply_threshold_filter(analysis_results["df_agg_moran_w"], threshold_N, threshold_type, SELECTION_COL)

In [ ]:
gdf_remap = gdf_threshold[gdf_threshold[SELECTION_COL] == 1.0]
gdf_remap[["remap_score"]].describe().loc[["count", "min"], :]

In [ ]:
gdf_threshold[SELECTION_COL].value_counts()

In [ ]:
create_task_map(gdf_threshold, "remap_score", ["task_id", "remap_score", "0_share_uw", SELECTION_COL], {"remap_score": "Remap Score", "0_share_uw": "Raw % 'No' Responses", "task_id": "Task ID", SELECTION_COL: "Selected for remapping?"}, selection_col=SELECTION_COL, color_bounds=(0.0, 1.0))